In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
print(train.head())
print(train.shape)
test = pd.read_csv('test.csv')
print(test.head())
print(test.shape)

                    qid                                      question_text  \
0  00002165364db923c7e6  How did Quebec nationalists see their province...   
1  000032939017120e6e44  Do you have an adopted dog, how would you enco...   
2  0000412ca6e4628ce2cf  Why does velocity affect time? Does velocity a...   
3  000042bf85aa498cd78e  How did Otto von Guericke used the Magdeburg h...   
4  0000455dfa3e01eae3af  Can I convert montra helicon D to a mountain b...   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
(1306122, 3)
                    qid                                      question_text
0  00014894849d00ba98a9  My voice range is A2-C5. My chest voice goes u...
1  000156468431f09b3cae           How much does a tutor earn in Bangalore?
2  000227734433360e1aae  What are the best made pocket knives under $20...
3  0005e06fbe3045bd2a92  Why would they add a hypothetical scenario tha...
4  00068a0f7f41f50fc399   What is the dresscode for Techmahindra freshe

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [8]:
from sklearn.model_selection import KFold

In [98]:
from gensim.models import KeyedVectors

c:\users\e6230\env\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Бейзлайн (0 - хорошие вопросы, 1 - плохие)

In [81]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=True)
X_train = vectorizer.fit_transform(train['question_text'])
print(X_train.shape)

(1306122, 25000)


In [89]:
kf = KFold(n_splits=3)

In [112]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [10]:
for train_index, test_index in kf.split(X_train):
    X_tr, X_te = X_train[train_index], X_train[test_index]
    y_tr, y_te = y_train[train_index], y_train[test_index]
    rf = RandomForestClassifier()
    rf.fit(X_tr, y_tr)
    y_train_pred = rf.predict(X_tr)
    y_test = rf.predict(X_te)
    print(classification_report(y_tr,y_train_pred))
    print(classification_report(y_te,y_test))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    816919
          1       1.00      0.89      0.94     53829

avg / total       0.99      0.99      0.99    870748

             precision    recall  f1-score   support

          0       0.95      1.00      0.97    408393
          1       0.71      0.17      0.27     26981

avg / total       0.93      0.94      0.93    435374

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    816728
          1       1.00      0.90      0.94     54020

avg / total       0.99      0.99      0.99    870748

             precision    recall  f1-score   support

          0       0.95      1.00      0.97    408584
          1       0.70      0.17      0.28     26790

avg / total       0.93      0.94      0.93    435374

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    816977
          1       1.00      0.90 

Определенные слова дают высокую точность, но низкую полноту. Похоже, они соответствуют какому-то одному типу "плохих" вопросов (например, оскорбительным). Хочется посмотреть на данные и понять,  есть ли в них какие-то нелексические признаки плохих вопросов.

In [11]:
bad = train.loc[train['target'] == 1]
print(bad)

                          qid  \
22       0000e91571b60c2fb487   
30       00013ceca3f624b09f42   
110      0004a7fcb2bf73076489   
114      00052793eaa287aff1e1   
115      000537213b01fd77b58a   
119      00056d45a1ce63856fc6   
127      0005de07b07a17046e27   
144      00068875d7c82a5bcf88   
156      0006ffd99a6599ff35b3   
167      00075f7061837807c69f   
168      00076debbd82860ca33a   
208      000983f5b226cca636f4   
214      0009fcb845bb24de91f4   
224      000a898565e80fe124bf   
235      000b1f4cbc5c238a765d   
260      000c44fe9b2822b25d77   
286      000d9de4df8d6a31c248   
289      000db4d0223af5dd9c5d   
291      000db9603d16a6d9bb2e   
302      000e67648fce55f011be   
315      000f0266e12c3349db07   
333      000f89527b907f3ef20f   
362      0010b8bd37af629de2f9   
397      0012bba774c13ce9cc9e   
400      0012d72ccc14e9329f97   
439      0014e0200bb280d44ea1   
460      0015e367aa794e5d547f   
496      00175f5e9c85b110614e   
538      00199a473cba7b54d974   
544      0

Видно, что задаватели плохих вопросов ставят больше вопросительных и восклицательных знаков

In [12]:
from collections import Counter

In [22]:
from tqdm import tqdm

In [83]:
punct_feature = []
for text in tqdm(train['question_text']):
    q = Counter(text).get('?',0)
    e = Counter(text).get('!',0)
    if q > 1 or e >1:
        punct_feature.append(1)
    else:
        punct_feature.append(0)

100%|█████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:28<00:00, 46373.15it/s]


In [55]:
from scipy.sparce import csr_matrix, hstack

In [86]:
X1 = hstack((X_train,csr_matrix(punct_feature).T))

In [114]:
X1=csr_matrix(X1)

In [87]:
X1.shape

(1306122, 25001)

In [115]:
for train_index, test_index in kf.split(X_train):
    X_tr, X_te = X1[train_index], X1[test_index]
    y_tr, y_te = y_train[train_index], y_train[test_index]
    lr = LogisticRegression()
    lr.fit(X_tr, y_tr)
    y_train_pred = lr.predict(X_tr)
    y_test = lr.predict(X_te)
    print(classification_report(y_tr,y_train_pred))
    print(classification_report(y_te,y_test))

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816919
          1       0.73      0.43      0.54     53829

avg / total       0.95      0.96      0.95    870748

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    408393
          1       0.71      0.41      0.52     26981

avg / total       0.95      0.95      0.95    435374

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816728
          1       0.73      0.43      0.54     54020

avg / total       0.95      0.96      0.95    870748

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    408584
          1       0.71      0.41      0.52     26790

avg / total       0.95      0.95      0.95    435374

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816977
          1       0.73      0.43 

Стало лучше. Попробуем еще эмбеддинги

In [99]:
model = KeyedVectors.load("word2vec.model")

In [96]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # еще раз код с лекции
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) 
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [105]:
dim = 200
X_w2v = np.zeros((len(train['question_text']), dim))

for i, text in enumerate(tqdm(train['question_text'])):
    X_w2v[i] = get_embedding(text, model, dim)

100%|██████████████████████████████████████████████████████████████████████| 1306122/1306122 [05:13<00:00, 4163.58it/s]


In [106]:
X_w2v.shape

(1306122, 200)

In [107]:
X2 = hstack((X1,csr_matrix(X_w2v)))

In [109]:
X2.shape

(1306122, 25201)

In [116]:
X2=csr_matrix(X2)

In [117]:
for train_index, test_index in kf.split(X_train):
    X_tr, X_te = X2[train_index], X2[test_index]
    y_tr, y_te = y_train[train_index], y_train[test_index]
    lr = LogisticRegression()
    lr.fit(X_tr, y_tr)
    y_train_pred = lr.predict(X_tr)
    y_test = lr.predict(X_te)
    print(classification_report(y_tr,y_train_pred))
    print(classification_report(y_te,y_test))

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816919
          1       0.73      0.43      0.54     53829

avg / total       0.95      0.96      0.95    870748

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    408393
          1       0.71      0.41      0.52     26981

avg / total       0.95      0.95      0.95    435374

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816728
          1       0.73      0.43      0.54     54020

avg / total       0.95      0.96      0.95    870748

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    408584
          1       0.71      0.41      0.52     26790

avg / total       0.95      0.95      0.95    435374

             precision    recall  f1-score   support

          0       0.96      0.99      0.98    816977
          1       0.73      0.43 

Похоже, что изменения в лучшую сторону  произошли от замены случайного леса на логрегрессию, а не от добавления признаков.

Можно много чего еще сделать: умную предобработку, убрать tf idf, но оставить эмбеддинги, попробовать какие-нибудь другие эмбеддинги, придумать более умные признаки. Но не в этом маленьком проекте.